In [13]:
import sys
from pathlib import Path

# Get project root (one level up from notebook/)
PROJECT_ROOT = Path.cwd().parent

# Add to Python path
sys.path.insert(0, str(PROJECT_ROOT))

print("Added to PYTHONPATH:", PROJECT_ROOT)

# sys.path[:3]


Added to PYTHONPATH: /Users/Harsh_Work/azure-openai-mcp-server


In [14]:
from dotenv import load_dotenv
load_dotenv()

from src.azure_openai_client import AzureOpenAIClient
from src.mcp_client import MCPToolRegistry


In [15]:
ai_client = AzureOpenAIClient()

tool_registry = MCPToolRegistry(
    file_base_path=str(PROJECT_ROOT / "test_files")
)


In [16]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant with access to tools."
    }
]


In [17]:
async def ask_llm(question, use_tools=True):
    conversation.append({
        "role": "user",
        "content": question
    })

    response = await ai_client.chat(
        message=question,
        tool_registry=tool_registry if use_tools else None,
        use_tools=use_tools,
        temperature=0.2
    )

    conversation.append({
        "role": "assistant",
        "content": response
    })

    return response


In [18]:
await ask_llm("Explain what MCP is in simple words", use_tools=False)


'Sure! MCP stands for **Microchannel Plate**.\n\nIn simple words, an MCP is a special kind of electronic device used to detect and amplify tiny particles like electrons or photons (light particles). Imagine it as a thin plate with lots of tiny holes (channels) inside it. When a particle enters one of these holes, it causes a chain reaction that creates many more particles, making the original signal much stronger and easier to detect.\n\nMCPs are used in things like night vision goggles, scientific instruments, and space telescopes to help "see" very faint signals that would otherwise be hard to detect.'

In [19]:
await ask_llm("What is 25 + 75 + 100?")


🔧 Calling tool: add with args: {'numbers': '25,75,100'}


'Let me calculate that for you:\n\n25 + 75 + 100 = 200\n\nSo, the answer is 200.'

In [20]:
await ask_llm("Convert 5 miles to kilometers")


🔧 Calling tool: convert_distance with args: {'value': 5, 'from_unit': 'mi', 'to_unit': 'km'}


'5 miles is equal to approximately 8.05 kilometers.'

In [21]:
await ask_llm("List all files in the directory")


🔧 Calling tool: list_files with args: {}


'Here are all the files in the directory:\n\n1. config.json\n2. data.txt\n3. ml_notes.txt\n4. python_info.txt\n5. sample.json\n6. shopping_list.txt\n7. test1.txt\n8. test2.txt\n\nLet me know if you want to view or search any of these files!'

In [22]:
await ask_llm("Read test1.txt")


🔧 Calling tool: read_file with args: {'filename': 'test1.txt'}


'Here is the content of test1.txt:\n\nThis is a sample document:\nOrganisms such as Indiaphonte bijoyi are essentially Meiofauna, tiny invertebrate animals less than 1 millimetre in size living in sediments in aquatic environments. With 350 species across 77 genera, the family Laophontidae is among the most diverse within the order Harpacticoida. Though microscopic in size, they play a vital role in maintaining the health of marine and freshwater ecosystems.\n\nLet me know if you need more information or want to read another file!'